In [1]:
from sklearn.cluster import KMeans
import numpy as np

def read_nodes_from_inp_file(filepath):
    """
    Read node data from an .inp file.
    
    Parameters:
    - filepath: str, path to the .inp file
    
    Returns:
    - nodes: dict, a dictionary where key is the z-coordinate and value is a list of tuples,
             each tuple contains node ID and x, y coordinates.
    """
    nodes = {}
    with open(filepath, 'r') as file:
        for line in file:
            # Assuming each relevant line is formatted as: "node_id, x, y, z"
            parts = line.strip().split(',')
            if len(parts) == 4:  # Check if the line is formatted correctly
                try:
                    # Convert the parts to the correct types
                    node_id = int(parts[0])
                    x, y, z = float(parts[1]), float(parts[2]), float(parts[3])
                    
                    # Group nodes by z-coordinate
                    if z not in nodes:
                        nodes[z] = []
                    nodes[z].append((node_id, x, y))
                except ValueError:
                    # Skip lines that cannot be converted properly
                    continue
    return nodes

def read_element_data(filepath):
    """
    Read element data from a file.
    
    Parameters:
    - filepath: str, path to the element data file
    
    Returns:
    - elements: dict, dictionary where keys are element IDs and values are lists of node IDs
    """
    elements = {}
    with open(filepath, 'r') as file:
        for line in file:
            # Strip leading/trailing whitespace and split the line by commas
            parts = [part.strip() for part in line.strip().split(',')]
            # Ensure the line has 1 element ID + 8 node IDs
            if len(parts) == 9:
                try:
                    # Convert element ID to int and node IDs to int, stripping spaces
                    element_id = int(parts[0])
                    node_ids = [int(node_id) for node_id in parts[1:]]
                    
                    # Store the node IDs list under the corresponding element ID key
                    elements[element_id] = node_ids
                except ValueError:
                    # Skip lines that cannot be converted properly
                    continue
    return elements



def find_elements_with_nodes_at_z(node_filepath, element_filepath, z_height):
    """
    Find and return element IDs that contain any node at the specified z-height.
    
    Parameters:
    - node_filepath: str, path to the node data file
    - element_filepath: str, path to the element data file
    - z_height: float, the specific z-coordinate to check nodes at
    
    Returns:
    - element_ids: list, list of element IDs that have nodes at the specified z-height
    """
    nodes_at_z = read_nodes_from_inp_file(node_filepath).get(z_height, [])
    elements = read_element_data(element_filepath)
    
    # Extract node IDs at the specified z-height
    node_ids_at_z = {node[0] for node in nodes_at_z}
    
    # Find elements that contain any of these node IDs
    element_ids = [element_id for element_id, node_ids in elements.items() if set(node_ids).intersection(node_ids_at_z)]
    
    return element_ids

def save_elements_with_nodes_at_z(node_filepath, element_filepath, z_height, output_file_path):
    """
    Save element IDs that contain any node at the specified z-height to a file.
    
    Parameters:
    - node_filepath: str, path to the node data file
    - element_filepath: str, path to the element data file
    - z_height: float, the specific z-coordinate to check nodes at
    - output_file_path: str, path to the output file where element IDs will be saved
    """
    element_ids = find_elements_with_nodes_at_z(node_filepath, element_filepath, z_height)
    
    with open(output_file_path, 'w') as file:
        for element_id in element_ids:
            file.write(f"Element ID: {element_id}\n")

    print(f"Elements with nodes at z={z_height} have been saved to {output_file_path}")

def read_grain_element_mapping(grain_filepath):
    """
    Read the grain to element mapping from a file.
    
    Parameters:
    - filepath: str, path to the grain-element mapping file
    
    Returns:
    - grains: dict, dictionary where keys are grain IDs (e.g., 'Elem1') and values are lists of element IDs
    """
    grains = {}
    current_grain = None
    with open(grain_filepath, 'r') as file:
        for line in file:
            # Check if the line starts a new grain definition
            if line.startswith('*Elset, elset='):
                # Extract the grain ID
                current_grain = line.split('=')[1].strip()
                grains[current_grain] = []
            elif current_grain and line.strip().isdigit():
                # Add the element ID to the current grain's list
                grains[current_grain].append(int(line.strip()))
    return grains

def find_grains_with_nodes_at_z_and_save(node_filepath, element_filepath, grain_filepath, z_height):
    """
    Find grains containing elements that have nodes at the specified z-height and save the grain IDs to a file.
    
    Parameters:
    - node_filepath: str, path to the node data file
    - element_filepath: str, path to the element data file
    - grain_filepath: str, path to the grain-element mapping file
    - z_height: float, the specific z-coordinate to check nodes at
    - output_file_path: str, path to the output file where the results will be saved
    """
    # Steps 1-4: Same as before, to identify grains with elements that have nodes at the specified z-height
    node_data = read_nodes_from_inp_file(node_filepath)
    node_ids_at_z = {node_id for z, nodes in node_data.items() if z == z_height for node_id, _, _ in nodes}

    element_data = read_element_data(element_filepath)
    elements_with_nodes_at_z = {elem_id for elem_id, node_ids in element_data.items() if node_ids_at_z.intersection(node_ids)}

    grain_data = read_grain_element_mapping(grain_filepath)
    grains_with_elements_at_z = [grain_id for grain_id, elem_ids in grain_data.items() if elements_with_nodes_at_z.intersection(elem_ids)]

    return grains_with_elements_at_z

def read_euler_angles_from_file(file_path):
    """
    Reads Euler angles from a text file and returns them as a numpy array.

    Parameters:
    - file_path: The path to the text file containing the Euler angles.

    Returns:
    - A numpy array of shape (M, 3) with the Euler angles.
    """
    # Adjust the delimiter based on the file format (e.g., ",", " ", "\t" for tab-separated values)
    return np.genfromtxt(file_path, delimiter=',', dtype=float, usecols=(0, 1, 2))


def assign_euler_angles_incrementally_with_clustering(node_filepath, element_filepath, grain_filepath, euler_angles_file, z_step, max_z, num_clusters):
    """
    Incrementally assigns Euler angles to grains at different z-heights, using clustering to maintain
    a representative distribution of orientations.
    
    Parameters:
    - node_filepath, element_filepath, grain_filepath: File paths for node, element, and grain data.
    - euler_angles_file: File path to the Euler angles data.
    - z_step: The increment step for z-height.
    - max_z: The maximum z-height to process.
    - num_clusters: Number of clusters to use in KMeans clustering.
    """
    euler_angles = read_euler_angles_from_file(euler_angles_file)
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(euler_angles)
    labels = kmeans.labels_

    # Dictionary to hold assigned Euler angles for each grain
    assigned_euler_angles = {}
    
    # Array to track which clusters have been used up
    cluster_usage_counts = np.zeros(num_clusters, dtype=int)

    for z_height in range(0, max_z + 1, z_step):
        grains = find_grains_with_nodes_at_z_and_save(node_filepath, element_filepath, grain_filepath, z_height)
        
        for grain_id in grains:
            if grain_id not in assigned_euler_angles:
                # Find the least used cluster to assign a new grain
                least_used_cluster = np.argmin(cluster_usage_counts)
                cluster_indices = np.where(labels == least_used_cluster)[0]
                
                # Select a random Euler angle from the cluster for the new grain
                selected_index = np.random.choice(cluster_indices)
                assigned_euler_angles[grain_id] = euler_angles[selected_index]
                
                # Update the usage count for the cluster
                cluster_usage_counts[least_used_cluster] += 1
        
    return assigned_euler_angles

def save_euler_angles_to_file(euler_angles, z_height):
    """
    Saves the given Euler angles to a .txt file named according to the z-height.

    Parameters:
    - euler_angles: A list of Euler angles to save.
    - z_height: The current z-height, which determines the file name.
    """
    file_name = f"euler_angles_z_{z_height}.txt"
    np.savetxt(file_name, euler_angles, delimiter=',')
    print(f"Saved Euler angles for z={z_height} to {file_name}")
    
def generate_grains_by_z_height(node_filepath, element_filepath, grain_filepath, start_z, max_z, z_step):
    """
    Generates a dictionary mapping z-heights to grain IDs present at those z-heights.
    
    Parameters:
    - node_filepath: Path to the node file.
    - element_filepath: Path to the element file.
    - grain_filepath: Path to the grain file.
    - start_z: The starting z-height.
    - end_z: The ending z-height.
    - z_step: The step size between z-heights to check.

    Returns:
    - A dictionary where keys are z-heights and values are lists of grain IDs at each z-height.
    """
    grains_by_z_height = {}
    
    for z_height in range(start_z, max_z + 1, z_step):
        grains = find_grains_with_nodes_at_z_and_save(node_filepath, element_filepath, grain_filepath, z_height)
        grains_by_z_height[z_height] = grains
    
    return grains_by_z_height

def save_grains_and_euler_angles_by_z_height(grains_by_z_height, assigned_euler_angles):
    """
    Saves grain IDs and their assigned Euler angles to .txt files, one file per z-height.

    Parameters:
    - grains_by_z_height: A dictionary mapping each z-height to a list of grain IDs at that height.
    - assigned_euler_angles: A dictionary mapping each grain ID to its Euler angles.
    """
    for z_height, grain_ids in grains_by_z_height.items():
        # Prepare the data to be saved for the current z-height
        data_for_saving = np.zeros((len(grain_ids), 4))
        for i, grain_id in enumerate(grain_ids):
            numeric_id = int(grain_id.replace('Elem', ''))
            euler_angles = assigned_euler_angles[grain_id]
            data_for_saving[i] = [numeric_id] + euler_angles.tolist()
        
        # Sort the data by the numeric grain ID (the first column)
        data_for_saving = data_for_saving[data_for_saving[:,0].argsort()]

        # Save the data to a .txt file named after the z-height
        file_name = f"/Users/janithwanniarachchi/Downloads/grains_and_euler_angles_z_{z_height}.txt"
        np.savetxt(file_name, data_for_saving, delimiter=',', fmt='%f')
        print(f"Saved grain IDs and Euler angles for z={z_height} to {file_name}")

# Paths to your files
node_filepath = '/Users/janithwanniarachchi/Downloads/Abaqus_nodes.inp'  # Replace with the path to your node data file
element_filepath = '/Users/janithwanniarachchi/Downloads/Abaqus_elems.inp'  # Replace with the path to your element data file
grain_filepath = '/Users/janithwanniarachchi/Downloads/Abaqus_elset.inp'  # Replace with actual path
euler_angles_file = '/Users/janithwanniarachchi/Downloads/50 ACS.txt'
out_put = '/Users/janithwanniarachchi/Downloads/50 ACS Euler.txt'
start_z = 0
z_step = 10  # The z-height increment
max_z = 100  # The maximum z-height to consider
num_clusters = 10  # The number of clusters for KMeans

assigned_euler_angles = assign_euler_angles_incrementally_with_clustering(
    node_filepath, element_filepath, grain_filepath, euler_angles_file, z_step, max_z, num_clusters)


# Sort the dictionary by grain ID and create a new ordered dictionary
sorted_assigned_euler_angles = {grain_id: assigned_euler_angles[grain_id] for grain_id in sorted(assigned_euler_angles)}

# Get the Grain ID at each height step
grains_by_z_height = generate_grains_by_z_height(node_filepath, element_filepath, grain_filepath, start_z, max_z, z_step)

# Saave the Euler angles and grain IDs at each height
save_grains_and_euler_angles_by_z_height(grains_by_z_height, assigned_euler_angles)

# Extract numeric part of grain IDs and combine with Euler angles
data_for_saving = np.zeros((len(assigned_euler_angles), 4))
for i, (elem_id, angles) in enumerate(assigned_euler_angles.items()):
    # Extract the numeric part of the grain ID
    numeric_id = int(elem_id.replace('Elem', ''))
    # Combine numeric ID with Euler angles
    data_for_saving[i] = [numeric_id] + angles.tolist()

# Sort the data by the numeric grain ID (the first column)
data_for_saving = data_for_saving[data_for_saving[:,0].argsort()]

# Save the Mx4 matrix to a text file
np.savetxt(out_put, data_for_saving, delimiter=',', fmt='%f')

print(f"Saved Euler angles with numeric IDs to {out_put}")




/Users/janithwanniarachchi/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Saved grain IDs and Euler angles for z=0 to /Users/janithwanniarachchi/Downloads/grains_and_euler_angles_z_0.txt
Saved grain IDs and Euler angles for z=10 to /Users/janithwanniarachchi/Downloads/grains_and_euler_angles_z_10.txt
Saved grain IDs and Euler angles for z=20 to /Users/janithwanniarachchi/Downloads/grains_and_euler_angles_z_20.txt
Saved grain IDs and Euler angles for z=30 to /Users/janithwanniarachchi/Downloads/grains_and_euler_angles_z_30.txt
Saved grain IDs and Euler angles for z=40 to /Users/janithwanniarachchi/Downloads/grains_and_euler_angles_z_40.txt
Saved grain IDs and Euler angles for z=50 to /Users/janithwanniarachchi/Downloads/grains_and_euler_angles_z_50.txt
Saved grain IDs and Euler angles for z=60 to /Users/janithwanniarachchi/Downloads/grains_and_euler_angles_z_60.txt
Saved grain IDs and Euler angles for z=70 to /Users/janithwanniarachchi/Downloads/grains_and_euler_angles_z_70.txt
Saved grain IDs and Euler angles for z=80 to /Users/janithwanniarachchi/Downloads/